In [1]:
## Mistral

In [1]:
pip install --upgrade accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
!pip install gdown

In [7]:
import gdown

# Replace with your actual Google Drive file ID or the full URL
FILE_ID = '1qzbTiJ2IL5ATZYNMp2DRkHvbFYsnOVAQ'
file_url = f'https://drive.google.com/uc?id={FILE_ID}'   # or use the full URL directly
output = 'filename.zip'  # Replace with the desired output filename

gdown.download(file_url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1qzbTiJ2IL5ATZYNMp2DRkHvbFYsnOVAQ
From (redirected): https://drive.google.com/uc?id=1qzbTiJ2IL5ATZYNMp2DRkHvbFYsnOVAQ&confirm=t&uuid=a8720019-552b-468c-86e0-374428970891
To: /content/filename.zip
100%|██████████| 250M/250M [00:03<00:00, 75.0MB/s]


'filename.zip'

In [8]:
!unzip /content/filename.zip

Archive:  /content/filename.zip
   creating: datasets/
  inflating: datasets/.DS_Store      
   creating: __MACOSX/
   creating: __MACOSX/datasets/
  inflating: __MACOSX/datasets/._.DS_Store  
   creating: datasets/beer/
  inflating: datasets/beer/test.txt  
  inflating: datasets/beer/test_label.txt  
  inflating: datasets/beer/train.txt  
   creating: datasets/restaurant/
  inflating: datasets/restaurant/test.txt  
  inflating: datasets/restaurant/test_label.txt  
  inflating: datasets/restaurant/train.txt  


In [9]:
from pathlib import Path
review_domain = 'restaurant'

input_dir = Path(f'/content/datasets/{review_domain}')
data_dir = Path('/content/datasets/')

In [10]:
import pandas as pd

def load_dataset(filename):
    f = open(filename, 'r', encoding='utf-8')
    all_reviews = f.readlines()
    print('Total Reviews: ', len(all_reviews))
    f.close()

    sentences = []
    for i,review in enumerate(all_reviews):
        sentences.append(review.strip('\n'))

    col = 'labels' if 'test_label' in str(filename) else 'text_org'

    df = pd.DataFrame({col:sentences})
    return df

df_label = load_dataset(input_dir/'test_label.txt')
print(df_label.shape)

Total Reviews:  3328
(3328, 1)


In [11]:


all_labels = list(set(df_label['labels'].str.split(' ').sum()))
print(all_labels)
drop_labels = ['Positive', 'Neutral']
all_labels = sorted([label for label in all_labels if label not in drop_labels])

drop_index = pd.Series(False, index=df_label.index)
for label in drop_labels:
    drop_index = drop_index | df_label['labels'].str.contains(label)

print(all_labels)

['Price', 'Food', 'Neutral', 'Miscellaneous', 'Staff', 'Ambience', 'Anecdotes', 'Positive']
['Ambience', 'Anecdotes', 'Food', 'Miscellaneous', 'Price', 'Staff']


In [12]:

import numpy as np
df_label = df_label[~drop_index].copy()
print(df_label.shape)

for label in all_labels:
    df_label[label] = np.where(df_label['labels'].str.contains(label), 1, 0)
print(df_label.shape)

df_label.head()

df = load_dataset(input_dir/'test.txt')
print(df.shape)

(3315, 1)
(3315, 7)
Total Reviews:  3328
(3328, 1)


In [13]:


df = df[~drop_index].copy()
print(df.shape)

df = pd.concat([df, df_label], axis=1)

df.head()

(3315, 1)


,text_org,labels,Ambience,Anecdotes,Food,Miscellaneous,Price,Staff
0,Always a fun place ... the food is deeelish !,Food Ambience,1,0,1,0,0,0
1,The staff is n't the friendliest or most comp...,Staff,0,0,0,0,0,1
2,"Great for groups , great for a date , great f...",Ambience,1,0,0,0,0,0
3,Another great place to take out-of-towners !,Miscellaneous,0,0,0,1,0,0
4,: ),Miscellaneous,0,0,0,1,0,0


In [14]:

df[all_labels].mean()

,0
Ambience,0.113424
Anecdotes,0.129412
Food,0.370136
Miscellaneous,0.276621
Price,0.089894
Staff,0.166817


In [15]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [16]:
# %pip install --upgrade accelerate
## Setup LLM: Mistral 7B

# !pip install transformers==4.20
# %pip install sentencepiece
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
repo_id = "Open-Orca/Mistral-7B-OpenOrca"

from transformers import AutoTokenizer, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(repo_id, torch_dtype="auto", low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained(repo_id, torch_dtype="auto", low_cpu_mem_usage=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [29]:
%pip install langchain langchain_core langchain_community kor tqdm

In [30]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFaceHub, HuggingFacePipeline
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 50,
    device=device,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0,
)

llm = HuggingFacePipeline(pipeline=pipe)

In [31]:
import langchain
from typing import List, Union, Optional
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.prompts.chat import ChatPromptTemplate,SystemMessagePromptTemplate,AIMessagePromptTemplate,HumanMessagePromptTemplate
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, Field, validator
import enum
from kor import extract_from_documents, from_pydantic
from kor.extraction import create_extraction_chain

## Setup Aspect Extractor

class Aspects(enum.Enum):
    staff = "Staff"
    ambience = "Ambience"
    anecdotes = "Anecdotes"
    food = "Food"
    price = 'Price'
    miscellaneous = 'Miscellaneous'

class ReviewABSA(BaseModel):
    aspect: List[Aspects] = Field(
        description=f"The key features that customers are talking about in their {review_domain} reviews.",
        examples=[
            ("The Singapore Mai Fun had NO curry flavor whatsoever.", "Food"),
            ("Go for it !", "Miscellaneous"),
            ("While the staff at this little bistro is very friendly , I have never experienced more incompetency.", "Staff"),
            ("Went on a double date with friend and his girlfriend for a few drinks and appetizers .", "Anecdotes"),
            ("When you enter , you feel like you have entered your mom 's kitchen .", "Ambience"),
            ("Pricey , but worth a try , at least once .", "Price")
        ],
        many=True,
    )

instruction_template = PromptTemplate(
    input_variables=["format_instructions", "type_description"],
    template=(
        "<|im_start|>system\nYour goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.\n\n"
        "{type_description}\n<|im_end|>\n<|im_start|>user\n{format_instructions}\n"
    ),
)

#from langchain.chains import create_extraction_chain
schema, validator = from_pydantic(ReviewABSA)
chain = create_extraction_chain(
    llm, schema, validator=validator,
    encoder_or_encoder_class="json",
    instruction_template=instruction_template
)

print(chain.get_prompts()[0].format_prompt(text="[user input]").to_string())

<|im_start|>system
Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

reviewabsa: { // 
 aspect: Array<"Staff" | "Ambience" | "Anecdotes" | "Food" | "Price" | "Miscellaneous"> // The key features that customers are talking about in their restaurant reviews.
}
```

<|im_end|>
<|im_start|>user
Please output the extracted information in JSON format. Do not output anything except for the extracted information. Do not add any clarifying information. Do not add any fields that are not in the schema. If the text contains attributes that do not appear in the schema, please ignore them. All output must be in JSON format and follow the schema specified above. Wrap the JSON in <json> tags.


Input: The Singapore Mai Fun had NO curry flavor whatsoever.
Output: <json>{"re

In [ ]:
%%time
from tqdm import tqdm
reviews = df['text_org'].values.tolist()
sample = 100
all_labels_sampled=all_labels[0:sample]
responses = []
aspects = []

for text in tqdm(reviews_sampled):
    response = chain.invoke(input=text+'<|im_end|>\n')
    responses.append(response)
    aspects.append(response['data']['reviewabsa']['aspect'] if len(response['data'].keys())>0
                   else ['Miscellaneous'])
reviews_sampled=reviews[0:sample]
aspects_sampled=aspects[0:sample]
len(reviews), len(responses), len(aspects)

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
  1%|          | 1/100 [00:09<16:08,  9.78s/it]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
  2%|▏         | 2/100 [00:17<13:38,  8.35s/it]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sam

In [35]:
## Aspect Extraction
df_test_pred = pd.DataFrame(columns=[f'predicted_{label}' for label in all_labels_sampled])
df_test_pred['predicted_labels'] = [' '.join(aspect) for aspect in aspects_sampled]
for label in all_labels_sampled:
    df_test_pred[f'predicted_{label}'] = np.where(df_test_pred['predicted_labels'].str.contains(label), 1, 0)
df_test_pred.index = df.iloc[0:sample].index
print(df_test_pred.shape)

(100, 7)


In [37]:
## Evaluate Results
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
df_final = pd.concat([df.iloc[0:sample], df_test_pred], axis=1)
print(df_final.shape)
df_final = df_final[['text_org', 'labels', 'predicted_labels']+all_labels_sampled+[f'predicted_{label}' for label in all_labels_sampled]]
df_final.head()

interested_labels = ['Ambience', 'Food', 'Price', 'Staff']
for label in interested_labels:
    print('Label: ', label)
    y_true = df_final[label]
    y_pred = df_final[f'predicted_{label}']
    print('Accuracy: ', np.round(accuracy_score(y_true, y_pred),3))
    print('F1 Score: ', np.round(f1_score(y_true, y_pred),3))
    print('Precision: ', np.round(precision_score(y_true, y_pred),3))
    print('Recall: ', np.round(recall_score(y_true, y_pred),3))
    print('***************')

(100, 15)
Label:  Ambience
Accuracy:  0.83
F1 Score:  0.0
Precision:  0.0
Recall:  0.0
***************
Label:  Food
Accuracy:  0.76
F1 Score:  0.0
Precision:  0.0
Recall:  0.0
***************
Label:  Price
Accuracy:  0.97
F1 Score:  0.0
Precision:  0.0
Recall:  0.0
***************
Label:  Staff
Accuracy:  0.81
F1 Score:  0.0
Precision:  0.0
Recall:  0.0
***************


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defin

In [38]:
other_labels = ['Anecdotes', 'Miscellaneous']
for label in other_labels:
    print('Label: ', label)
    y_true = df_final[label]
    y_pred = df_final[f'predicted_{label}']
    print('Accuracy: ', np.round(accuracy_score(y_true, y_pred),3))
    print('F1 Score: ', np.round(f1_score(y_true, y_pred),3))
    print('Precision: ', np.round(precision_score(y_true, y_pred),3))
    print('Recall: ', np.round(recall_score(y_true, y_pred),3))
    print('***************')

Label:  Anecdotes
Accuracy:  0.82
F1 Score:  0.0
Precision:  0.0
Recall:  0.0
***************
Label:  Miscellaneous
Accuracy:  0.33
F1 Score:  0.496
Precision:  0.33
Recall:  1.0
***************


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
